# Aqua 0.7:Quantum Optimization Design Doc

| **Status**        | **Proposed** |
|:------------------|:---------------------------------------------|
| **RFC #**         | ####                                         |
| **Authors**       | Stefan Woerner (wor@zurich.ibm.com)          |
|                   | Julien Gacon (jul@zurich.ibm.com)            |
| **Submitted**     | 2020-01-30                                   |
| **Updated**       | YYYY-MM-DD                                   |

# Motivation

### Current Situation

#### Current restriction to QUBOs
Currently, Qiskit only supports Quadratic Unconstrained Binary Optimization (QUBO) problems.
QUBOs can be mapped to an equivalent operator ground state problem.
The corresponding algorithms available in Qiskit (VQE / QAOA) expect an 'Operator' in the constructor that specifies the problem to be solved.

For some problem classes (e.g. TSP, MaxCut, Portfolio, ...), Qiskit provides dedicated translators that take problem parameters and return the corresponding operator.
In addition, a more general translator from a `docplex.mp.model.Model` to an `Operator` is available.

In principle, `docplex.mp.model.Model` can represent more general types of (quadratic) optimization problems, but since the available algorithms require an `Operator` as input, we are again limited to QUBOs.

#### Missing model flexibility for advanced algorithms

In addition, `docplex.mp.model.Model` models are very convenient to formulate a problem, but not flexible at all once this is done.
E.g., it is not possible to substitute particular variables by given values and return a smaller optimization problem, which is a very important functionality for many of our more advanced algorithms (e.g. ADMM, RQAOA, Slack-MBO).


### Creating a modular and flexible optimization stack

Our goal is to increase flexibility and modularity of optimization problems and algorithms.
This requries a unified `OptimizationProblem` representation that satisfies the following criteria:

- Easy and intuitive developement of complex optimization problems using a high-level modelling language (e.g. docplex)
- Flexible and easy to modify problem representation (e.g. substituting variables, coefficients, etc.)
- Efficient scaling to large problems
- Generic representation to cover a large set of problem classes

Furthermore, we want all `OptimizationAlgorithms` to implement the same interface, i.e., accept an `OptimizationProblem` and return (an extension of) a generic `Solution`.
An algorithm should throw an exception if it is given an unsupported `OptimizationProblem` (common practise in classical optimization).

# Proposal & Possible implementations

In the following, we outline the general idea for a modular and flexible optimization stack.
This supports quantum (pure & hybrid) but also classical algorithms, and by streamlining the interfaces supports efficient modeling, testing, validation, benchmarking, etc.

We suggest to split the stack into three classes, each responsible for a well-defined task:

- `OptimizationProblem` for specifying, representing, modifying the problem to be solved
- `OptimizationAlgorithm` to find/approximate the optimal solution of an `OptimizationProblem`
- `Solution` to return (intermediate) results of an `OptimizationAlgorithm` while trying to solve a `OptimizationProblem`


### Folder Structure

We suggest the following organization for the code:
```
qiskit/optimization/
qiskit/optimization/problems     # contains `OptimizationProblem` and classes used therein
qiskit/optimization/solutions    # contains `Solution` and related quality metrics
qiskit/optimization/algorithms   # contains `OptimizationAlgorithm` and derived algorithms
qiskit/optimization/converters   # contains converters to manipulate given Optimization problems, e.g.,
                                 # - mapping integer to binaries variables
                                 # - adding linear equality constraints as penalty terms to the objective
                                 # - translating a QUBO given as OptimizationProblem to an operator
qiskit/optimization/translators  # problem-specific translators, e.g., for
                                 # - graph problems (TSP, MaxCut, ...)
                                 # - portfolio optimization / diversification
qiskit/optimization/utils        # helper classes / methods
```

### Optimization Problem

We limit ourselves to Quadratically Constrained Quadratic Mixed Integer Programs.
Although this is a restriction, this is still a very powerful problem class that adopts many relevant applications.
Many commercial solvers make this restriction as it enables efficient scaling to larger problems and sufficient structure to be exploited in optimization algorithms.
It can also be extended to Semidefinite Programs, which we plan to do at a later stage.
Particular problem classes that will eventually be covered are, e.g., QUBO, MIP, LP, SDP.

The `OptimizationProblem` should follow the CPLEX interface (https://pypi.org/project/cplex/) and internally represent objective, constraints etc. using (sparse) matrices and vectors and lists thereof.
This has the advantage that

- it scales efficiently to large problems
- we can reuse many existing code written for CPLEX or allows straight-forward conversion (e.g. to convert from docplex/cplex, to use CPLEX as solver for validation/testing/benchmarking)
- many people in the optimization community are already familiar with it
- the representation using matrices / vectors allows to efficiently substitute variables by constants or other (scaled) variables, which is crucial for developing new algorithms

Constructing a problem can then be done, e.g., directly using `OptimizationProblem` (following CPLEX API):

In [ ]:
# initialize problem
op = OptimizationProblem()

# add variables
op.variables.add(names=["x0", "x1", "x2", "x3"], types='B'*4)

# set offset
op.objective.set_offset(1.0)

# set linear terms of objective function (uses names as well as indices)
op.objective.set_linear([("x0", 2.0), (1, 0.5)])
op.objective.set_linear("x2", -1.0)
op.objective.set_linear(3, -1.0)

# set quadratic part of objective
op.objective.set_quadratic([SparsePair(ind=[0, 1, 2], val=[1.0, -2.0, 0.5]),
                            SparsePair(ind=[0, 1, 2], val=[-2.0, -1.0, 1.0]),
                            SparsePair(ind=[0, 1, 2], val=[0.5, 1.0, -3.0])])

# set objective sense (maximize/minimize)
op.objective.set_sense(op.objective.sense.maximize)

Alternatively, we can use automatically translating from `cplex.Cplex` or `docplex.mp.model.Model` or back:

In [ ]:
# some code to create docplex / cplex models
docplex = [...]
cplex = [...]

# load from docplex
op1 = OptimizationProblem()
op1.from_docplex(docplex)

# load from cplex
op2 = OptimizationProblem()
op2.from_cplex(cplex)

# convert to docplex / cplex
docplex = op1.to_docplex()
cplex = op2.to_cplex()

### Solution

A `Solution` object should contain the following fields (but very likely a lot more, depending on the algorithm):

- x (= final solution)
- f_mean
- f_stddev
- best_found_solution
- best_bound
- num_iters
- termination_criterion
- history (intermediate results for iterative algorithms)
- ...

The `Solution` object could be based on the CPLEX Solution class as well (or a subset), to leverage existing conventions. However, this needs to be developed in parallel to the algorithms, as it is difficult to see all possibilities upfront.

### Optimization Algorithms

- `OptimizationAlgorithm` provides a standard interface to solve `OptimizationProblem` (should also accept `cplex.Cplex` and `docplex.mp.model.Model` and cast it internally)  and return a `Solution` object.
- Algorithm properties are provided via the initializer or by setting parameters after construction.
- Algorithms are usually only compatible with some problem types and should throw a corresponding exception (common behavior of optimization algorithms) in case of facing an unsupported problem.
- This covers quantum as well as classical algortihms that can be easily interchanged for testing, validation, or benchmarking, e.g., we want to be able to pass the same problem to a `CplexWrapper`, to `QAOA`, or `GroverAdaptiveSearch`.
- Having a standard interface also allows to easily build meta-algorithms that expect one or more different `OptimizationAlgorithm` for a certain problem type as sub-routine (e.g. ADMM or RQAOA)

`OptimizationAlgorithm` could be used as follows.

In [ ]:
# initialize algorithm with parameters
alg = OptimizationAlgorithm(parameters)
result = alg.solve(op)

Existing algorithms from core Aqua such as VQE can be used as sub-routines in optimization algorithms which provide the required translations, e.g. from optimization problem to operator (VQE/QAOA) or oracles (Grover Adaptive Search), as the following example illustrates. 

In [ ]:
class OperatorEigenSolver(OptimizationAlgorithm):

    def __init__(self, eigen_solver: MinEigenSolver, ...):
        self._eigen_solver = eigen_solver
        
    def solve(self, op: OptimizationProblem):
        operator, offset = translate_problem(op)
        eigen_results = self._eigen_solver.solve(operator)
        solution = translate_results(min_eigen_results, offset)
        return solution

An example that illustrates the power of this design is ADMM (https://arxiv.org/abs/2001.02069), which accepts problems consisting of continuous and binary decision variables as well as constraints (plus some assumptions on the structure) and in each iteration decomposes the problem into a convex continuous problem as well as a QUBO. 
Thus, ADMM requires a solver for convex continuous problems (which may be classical) plus a QUBO solver (which may be quantum).
Given the suggested design, we can, for instance, easily use CPLEX, QAOA, or Grover Adaptive Search as QUBO solver and compare their performance or validate the results.

In [ ]:
class ADMM(OptimizationAlgorithm):
    
    def __init__(self, convex_solver: OptimizationAlgorithm, 
                 qubo_solver: OptimizationAlgorithm, 
                 ...):
        self._convex_solver = convex_solver
        self._qubo_solver = qubo_solver
        
    def solve(self, op: OptimizationProblem):
        ...
        qubo_result = self._qubo_solver.solve(qubo)
        cvx_result = self._convex_solver.solve(cvx)
        ...

In [ ]:
# define convex solver
convex_solver = ConvexSolver(...)

# define QUBO solver
rqaoa = RQAOA(...)
gas   = GroverAdaptiveSearch(...)
cplex = CPLEX(...)

# construct different versions of ADMM
admm_rqaoa = ADMM(convex_solver, rqaoa, ...)
admm_gas   = ADMM(convex_solver, gas, ...
admm_cplex = ADMM(convex_solver, cplex, ...)

# test different version on problem
res_rqaoa = admm_rqaoa.solve(op)
res_gas   = admm_gas.solve(op)
res_cplex = admm_cplex.solve(op)